In [1]:
# --- 0. 安装必要的库 ---
# LDA 需要 'gensim'；交互式可视化需要 'pyLDAvis'
!pip install -q gensim pyLDAvis

# --- 1. 导入库 ---
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm
import os
import sys

# gensim
try:
    from gensim.corpora import Dictionary
    from gensim.models import LdaModel
except ImportError:
    print("错误：无法导入 gensim，请先成功安装。")
    sys.exit()

# pyLDAvis
try:
    import pyLDAvis
    import pyLDAvis.gensim_models as gensim_vis
    pyLDAvis.enable_notebook()
except ImportError:
    print("错误：无法导入 pyLDAvis，请先成功安装。")
    sys.exit()

# --- 2. 设置字体与文件路径 ---
font_path_chinese = "/kaggle/input/nsfc2025-management/simkai.ttf"
font_path_english = "/kaggle/input/nsfc2025-management/times.ttf"
data_file = "/kaggle/input/nsfc2025-management/nsfc-2025-management-title-fenci.csv"

# 字体注册（pyLDAvis主要在浏览器端渲染，但保留注册以备其他绘图使用）
if os.path.exists(font_path_chinese):
    fm.fontManager.addfont(font_path_chinese)
if os.path.exists(font_path_english):
    fm.fontManager.addfont(font_path_english)

# --- 3. 加载与准备数据（仅两列） ---
print(f"正在从 '{data_file}' 加载数据...")
try:
    df = pd.read_csv(data_file, dtype=str, encoding="utf-8")
except FileNotFoundError:
    print(f"错误: 数据文件 '{data_file}' 未找到。")
    sys.exit()
except Exception as e:
    print(f"读取 CSV 时出错: {e}")
    sys.exit()

# 统一去除列名两端空格，兼容“项目名称分词 ”
df.columns = [str(c).strip() for c in df.columns]
required_cols = {"项目名称", "项目名称分词"}
if not required_cols.issubset(set(df.columns)):
    print(f"错误：数据中缺少必需列。需要列：{required_cols}，当前列：{list(df.columns)}")
    sys.exit()

# 仅保留两列并清洗
df = df.loc[:, ["项目名称", "项目名称分词"]].dropna(subset=["项目名称分词"]).copy()

# 原始分词文本（列表形式）
texts_raw = df["项目名称分词"].apply(lambda x: str(x).split()).tolist()
if not texts_raw:
    print("错误：分词文本为空。")
    sys.exit()

print(f"共找到 {len(texts_raw)} 篇文档。")

# --- 4. 文本预处理（移除停用词，可按需增删） ---
my_stop_words = {'研究', '机制', '基于', '影响', '优化', '效应', '视角', '策略', 
                 '驱动', '理论', '机理', '路径', '动态', '分析', '应用', '作用', 
                 '模型', '方法', '评估', '及其', '设计', '中国', '模式', '形成', 
                 '发展', '结构', '系统', '背景', '提升', '后果', '实证', '考虑', 
                 '问题', '建模', '面向', '测度', '效果', '预测', '识别', '构建', 
                 '关键', '传导', '——','--'}
print(f"将从文本中移除 {len(my_stop_words)} 个自定义停用词...")

texts_processed = [[w for w in doc if w not in my_stop_words] for doc in texts_raw]
if not any(texts_processed):
    print("错误：移除停用词后所有文档均为空。")
    sys.exit()
print("停用词移除完成。")

# --- 5. 创建词典与语料库 ---
print("正在创建 Gensim 词典和语料库...")
id2word = Dictionary(texts_processed)
# 过滤极端词：出现文档数 <2 的词去除；在 >50% 文档中出现的高频词去除
id2word.filter_extremes(no_below=2, no_above=0.5)
corpus = [id2word.doc2bow(text) for text in texts_processed]
if not any(corpus):
    print("错误：语料库为空，可能是过滤条件过于严格。")
    sys.exit()
print(f"词典和语料库创建完成。过滤后词典大小：{len(id2word)}")

# --- 6. 训练最终 LDA 模型（主题数=12） ---
num_topics = 12
print(f"正在训练 LDA 模型，主题数 K={num_topics} ...")
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=num_topics,
    random_state=2025,
    passes=15,        # 增加训练遍数以提高稳定性
    chunksize=100
)
print("LDA 模型训练完成。")

# --- 7. 准备并保存 pyLDAvis 交互式可视化 ---
print("正在准备 pyLDAvis 交互式可视化 ...")
vis = gensim_vis.prepare(lda_model, corpus, id2word)

output_filename = f"lda_k{num_topics}_interactive.html"
try:
    pyLDAvis.save_html(vis, output_filename)
    print(f"交互式图表已成功保存为: {output_filename} (位于 /kaggle/working/ 目录)")
except Exception as e:
    print(f"保存交互式 HTML 时出错: {e}")

# --- 8. 在 Notebook 中显示交互式图表 ---
print("\n" + "="*50)
print("正在 Notebook 中显示交互式图表 ...")
print("="*50)
vis  # 在单元格末尾返回 vis 对象以渲染

# --- 9. 打印每个主题的 Top 10 主题词 ---
print("\n" + "="*50)
print(f"主题词分布（Top 10 per topic, K={num_topics}）")
print("="*50)
for topic_id in range(num_topics):
    print(f"\n【主题 {topic_id}】")
    for rank, (term, prob) in enumerate(lda_model.show_topic(topic_id, topn=10), start=1):
        print(f"{rank:>2}. {term:<15s} 概率: {prob:.4f}")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 33.2 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.2.2 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1